In [2]:
from wikibaseintegrator.models import Qualifiers, References, Reference
from wikibaseintegrator import WikibaseIntegrator
from wikibaseintegrator import wbi_login
from wikibaseintegrator import datatypes
from wikibaseintegrator.wbi_config import config as wbi_config
from os.path import join, isfile, exists, isdir, abspath, pardir
from os import listdir, getcwd
import pandas as pd
from wikibaseintegrator.datatypes import (URL, CommonsMedia, ExternalID, Form, GeoShape, GlobeCoordinate, Item, Lexeme, Math, MonolingualText, MusicalNotation, Property, Quantity, Sense, String, TabularData, Time)
from wikibaseintegrator.wbi_enums import ActionIfExists, WikibaseDatePrecision, WikibaseRank, WikibaseSnakType
from wikibaseintegrator.wbi_exceptions import MissingEntityException, ModificationFailed, MWApiError

In [4]:
# Default variables
WDUSER, WDPASS = "Admin", "90J8XXXaO4Sr9^^Z"
# WDUSER, WDPASS = "Mohammadzainabbas", "fHh!%shFa6^h"

In [26]:
parent_dir = abspath(join(getcwd(), pardir))
data_dir = join(parent_dir, "data")
team_data = join(data_dir, "BDMA_Team_v1.csv")
project_data = join(data_dir, "BDMA_Projects_v1.csv")
column_mapping_data = join(data_dir, "column_mapping.csv")

In [5]:
def setup_config():
    """
    Set up WBI config to use local docker installation
    """
    wbi_config['MEDIAWIKI_API_URL'] = 'http://139.144.66.193:8181/api.php'
    wbi_config['SPARQL_ENDPOINT_URL'] = '"http://139.144.66.193:8282/proxy/wdqs/bigdata/namespace/wdq/sparql"'
    wbi_config['WIKIBASE_URL'] = 'http://wikibase.svc'

In [6]:
setup_config()

In [7]:
login = wbi_login.Login(user=WDUSER, password=WDPASS)

In [8]:
wbi = WikibaseIntegrator(login=login)

In [27]:
df = pd.read_csv(column_mapping_data)
column_mapping = df.set_index("column").T.to_dict()

In [16]:
team_df = pd.read_csv(team_data)
team_df.head()

,Id,Name,Family Name,Birthdate,Nationality,Category,Start date,End date
0,S000001,Mohammad Zain,Abbas,09/05/1995,Pakistani,SK-N3,19/07/2022,06/09/2022
1,S000002,Kainaat,Amjid,30/12/1996,Pakistani,SK-N1,19/07/2022,13/09/2022
2,S000003,Sergio,Postigo,12/02/1996,Peru,SK-N2,26/07/2022,13/10/2022
3,S000004,Emmanuel,Jamin,28/09/1978,French,SKL-N2,10/01/2015,11/01/2015


In [17]:
project_df = pd.read_csv(project_data)
project_df.head()

,Client Type,Client,Project,Project Type,Task,Owner,Member,Duration (in days),Project Start date,Project End date
0,NTTData,SEMBU Team,SEMBU,Project,Wikibase,Emmanuel Jamin,Mohammad Zain Abbas,47,19/07/2022,06/09/2022
1,NTTData,SEMBU Team,SEMBU,Project,Solid Pods,Emmanuel Jamin,Kainaat Amjid,55,19/07/2022,13/09/2022
2,NTTData,SEMBU Team,SEMBU,Project,Entity Resolution,Emmanuel Jamin,Sergio Postigo,60,26/07/2022,28/09/2022


In [18]:
team_df['full_name'] = team_df['Name'] + " " + team_df['Family Name']

In [22]:
member_df = team_df.merge(project_df, left_on="full_name", right_on="Member", how="right")
member_df.drop(columns=["full_name"], inplace=True)
member_df.head()

,Id,Name,Family Name,Birthdate,Nationality,Category,Start date,End date,Client Type,Client,Project,Project Type,Task,Owner,Member,Duration (in days),Project Start date,Project End date
0,S000001,Mohammad Zain,Abbas,09/05/1995,Pakistani,SK-N3,19/07/2022,06/09/2022,NTTData,SEMBU Team,SEMBU,Project,Wikibase,Emmanuel Jamin,Mohammad Zain Abbas,47,19/07/2022,06/09/2022
1,S000002,Kainaat,Amjid,30/12/1996,Pakistani,SK-N1,19/07/2022,13/09/2022,NTTData,SEMBU Team,SEMBU,Project,Solid Pods,Emmanuel Jamin,Kainaat Amjid,55,19/07/2022,13/09/2022
2,S000003,Sergio,Postigo,12/02/1996,Peru,SK-N2,26/07/2022,13/10/2022,NTTData,SEMBU Team,SEMBU,Project,Entity Resolution,Emmanuel Jamin,Sergio Postigo,60,26/07/2022,28/09/2022


In [31]:
project_columns = ['Client Type', 'Client', 'Project', 'Project Type', 'Task', 'Owner']
project_df = member_df[project_columns]
project_df.head()

,Client Type,Client,Project,Project Type,Task,Owner
0,NTTData,SEMBU Team,SEMBU,Project,Wikibase,Emmanuel Jamin
1,NTTData,SEMBU Team,SEMBU,Project,Solid Pods,Emmanuel Jamin
2,NTTData,SEMBU Team,SEMBU,Project,Entity Resolution,Emmanuel Jamin


In [32]:
org_item = wbi.item.new(title="NTTData")

In [34]:
org_item.labels.set("en", "NTTData")
org_item.aliases.set("en", ["NTT Data", "NTT DATA"])
org_item.descriptions.set("en", "NTT Data Corporation is a Japanese multinational information technology services and consulting company headquartered in Tokyo, Japan.")
org_item.sitelinks.set("enwiki", "NTT Data")

<Sitelink @5ae8f0 site='enwiki' title='NTT Data' badges=[]>

In [36]:
org_item.add_claims([Property("P31", Item("Q4830453"))])

TypeError: Property.__init__() takes from 1 to 2 positional arguments but 3 were given

In [ ]:
new_qualifiers = Qualifiers()

new_qualifiers.add("P31", "Q5", snak_type=WikibaseSnakType.VALUE)

In [33]:
org_item.get_json()

{'labels': {}, 'descriptions': {}, 'aliases': {}, 'type': 'item', 'claims': {}}

In [29]:
column_mapping['Member']

{'prop': 'P15', 'data_type': 'string', 'type': 'internal', 'mapping_code': nan}

In [ ]:
item = wbi.item.new()

# iterate over dataframe rows
for index, row in member_df.iterrows():
    # get the item
    item = wbi.item.new()
    new_qualifiers = Qualifiers()
    new_references = References()
    is_claim = False

    # iterate over columns
    for column in row.index:
        __value = row[column]
        __col_mapping = column_mapping[column]

        if column == "Member":
            item.labels.set('en', __value)
            item.descriptions.set('en', 'A BDMA Team Member')
            item.sitelinks.set('enwiki', __value)
        elif column == "Project":
            item.labels.set('en', __value)
            item.descriptions.set('en', 'A BDMA Project')
            item.sitelinks.set('enwiki', __value)
        

    item.write(login)
        



        # get the wikibase property

